In [1]:
# Disables generation of pycache file
import sys
sys.dont_write_bytecode = True

import numpy as np

from solver import *

In [2]:
Q = np.array([180, 200, 140, 80, 180])
D = np.array([89, 95, 121, 101, 116, 181])
C = np.array([
    [4.50, 5.09, 4.33, 5.96, 1.96, 7.30],
    [3.33, 4.33, 3.38, 1.53, 5.95, 4.01],
    [3.02, 2.61, 1.61, 4.44, 2.36, 4.60],
    [2.43, 2.37, 2.54, 4.13, 3.20, 4.88],
    [6.42, 4.83, 3.39, 4.40, 7.44, 2.92]
])

opt_tol = 10**(-5)
time_limit = 7200

X, sensitivity_analysis, dual_variables = gurobi_solver(C, Q, D, opt_tol, time_limit)

print(X)

print(f"Objective value: {np.trace(np.dot(C.T, X))}")

print("Sensitivity analysis:")
print("Minimum_production_lower")
print(sensitivity_analysis["Minimum_production_lower"])
print("Minimum_production_upper")
print(sensitivity_analysis["Minimum_production_upper"])
print("Maximum_production_lower")
print(sensitivity_analysis["Maximum_production_lower"])
print("Maximum_production_upper")
print(sensitivity_analysis["Maximum_production_upper"])
print("Minimum_demand_lower")
print(sensitivity_analysis["Minimum_demand_lower"])
print("Minimum_demand_upper")
print(sensitivity_analysis["Minimum_demand_upper"])
print("Optimal_basis_lower")
print(sensitivity_analysis["Optimal_basis_lower"])
print("Optimal_basis_upper")
print(sensitivity_analysis["Optimal_basis_upper"])

print("Dual variables:")
print("Minimum_production")
print(dual_variables["Minimum_production"])
print("Maximum_production")
print(dual_variables["Maximum_production"])
print("Minimum_demand")
print(dual_variables["Minimum_demand"])


Set parameter TimeLimit to value 7200
Set parameter OptimalityTol to value 1e-05
[[ 19.   0.   0.   0. 116.   0.]
 [ 66.   0.   0. 101.   0.   1.]
 [  0.  19. 121.   0.   0.   0.]
 [  4.  76.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 180.]]
Objective value: 1651.02
Sensitivity analysis:
Minimum_production_lower
[116.0, -inf, -inf, -inf, -inf]
Minimum_production_upper
[153.0, 168.0, 140.0, 80.0, 180.0]
Maximum_production_lower
[135.0, 168.0, 136.0, 76.0, 148.0]
Maximum_production_upper
[inf, inf, 158.0, 98.0, 181.0]
Minimum_demand_lower
[71.0, 77.0, 103.0, 83.0, 98.0, 180.0]
Minimum_demand_upper
[121.0, 99.0, 125.0, 133.0, 135.0, 213.0]
Optimal_basis_lower
[array([[ 3.33,  4.44,  3.44,  2.7 ,  1.17,  5.18],
       [ 2.67,  3.27,  2.27,  0.  ,  0.79,  2.92],
       [ 2.67,  1.72, -0.66,  0.87,  0.13,  3.35],
       [ 1.78,  2.02,  1.37,  0.63, -0.11,  3.11],
       [ 2.24,  2.18,  1.18,  0.44, -0.3 ,  -inf]])]
Optimal_basis_upper
[array([[5.15,  inf,  inf,  inf, 4.19,  inf],
      

In [3]:
m, n = C.shape
q = Q.shape[0]
max_gain = 0
best_bound = ""
best_prod = ""
best_idx = -1

for i in range(0, q):
    for bound in ["lower", "upper"]:
        for prod in ["Minimum_production", "Maximum_production"]:
            if prod == "Minimum_production":
                # b = Q[i] + sensitivity_analysis[f"{prod}_{bound}"][i]
                b = sensitivity_analysis[f"{prod}_{bound}"][i]
                gain = - b * dual_variables[prod][i]
                if gain < max_gain:
                    max_gain = gain
                    best_bound = bound
                    best_prod = prod
                    best_idx = i
            else:
                # b = Q[i] + sensitivity_analysis[f"{prod}_{bound}"][i]
                b = sensitivity_analysis[f"{prod}_{bound}"][i]
                gain = 0.75 * b * dual_variables[prod][i]
                if gain < max_gain:
                    max_gain = gain
                    best_bound = bound
                    best_prod = prod
                    best_idx = i

print("Results:")
print("Best gain: ", max_gain)
print("Best bound: ", best_bound)
print("Best prod: ", best_prod)
print("Best idx: ", best_idx)

Results:
Best gain:  -179.01
Best bound:  upper
Best prod:  Minimum_production
Best idx:  0


In [14]:
sensitivity_analysis[f"Optimal_basis_lower"][0]

array([[ 3.33,  4.44,  3.44,  2.7 ,  1.17,  5.18],
       [ 2.67,  3.27,  2.27,  0.  ,  0.79,  2.92],
       [ 2.67,  1.72, -0.66,  0.87,  0.13,  3.35],
       [ 1.78,  2.02,  1.37,  0.63, -0.11,  3.11],
       [ 2.24,  2.18,  1.18,  0.44, -0.3 ,  -inf]])

In [22]:
best_delta = np.inf
best_i = -1
best_j = -1

for i in range(0, m):
    for j in range(0, n):
        for bound in ["lower", "upper"]:
            delta = -sensitivity_analysis[f"Optimal_basis_{bound}"][0][i, j] * X[i, j]
            if delta < best_delta:
                best_delta = delta
                best_i = i
                best_j = j

print(best_delta)
print(best_i)
print(best_j)
print(sensitivity_analysis[f"Optimal_basis_{bound}"][0][best_i, best_j])
print(X[best_i, best_j])
print(sensitivity_analysis[f"Optimal_basis_{bound}"][0][best_i, best_j] * X[best_i, best_j])

-721.8
4
5
4.01
180.0
721.8


/tmp/ipykernel_1593/2399591046.py:8: RuntimeWarning: invalid value encountered in scalar multiply
  delta = -sensitivity_analysis[f"Optimal_basis_{bound}"][0][i, j] * X[i, j]
